<a href="https://colab.research.google.com/github/cykrr/ada/blob/master/Optimal%20BST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
#@markdown Instalar Cairo (librería de dibujo vectorial)

#@markdown [https://www.cairographics.org](https://www.cairographics.org)
!apt-get install libcairo2-dev libjpeg-dev libgif-dev
!pip install pycairo

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libjpeg-dev is already the newest version (8c-2ubuntu8).
libcairo2-dev is already the newest version (1.15.10-2ubuntu0.1).
libgif-dev is already the newest version (5.1.4-2ubuntu0.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 12 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
#@markdown importar librerías necesarias. 

import cairo
from google.colab import files
from IPython.display import SVG, display, Image
from random import randint
from math import sin, cos, pi, inf
from copy import copy, deepcopy

In [17]:
#@markdown ## Clase TreeNode. 

#@markdown Utilizada para facilitar el retorno y la graficación de un (O)BST.

#@markdown  ### Parametros
#@markdown  `key`: LLave del nodo a almacenar. Valor predeterminado: 0

#@markdown  `freq`: Frequencia de búsqueda del nodo a almacenar.
#@markdown  Valor predeterminado: 0

#@markdown  ### Métodos (públicos)
#@markdown  `append`: Inserta un TreeNode `node` en el árbol

class TreeNode:
    def __init__(self, key = 0, freq = 0):
        self.key = key;
        self.freq = freq;
        
        self.left = None;
        self.right = None;
          
    def __set_right(self, node):
        self.right = node;

    def __set_left(self, node):
        self.left = node;

    def append(self, node):
        if node == None:
            raise Exception("Error. El nodo a agregar es None")
        if node.key == None:
            raise Exception("Error. El nodo a agregar no tiene llave " + 
                            "establecida")
        if node.key >= self.key:
            if self.right == None:
                self.__set_right(node);
            else:
                self.right.append(node);
        elif node.key < self.key:
            if self.left == None:
                self.__set_left(node);
            else:
                self.left.append(node);

In [18]:
#@markdown # Clase `OptimalValue`
#@markdown Utilizada para emparejar un  `TreeNode` con un
#@markdown costo óptimo. Vale decir, una tupla amigable.

#@markdown ## Parámetros

#@markdown `node`: TreeNode a almacenar.

#@markdown `optimal_cost`: Costo mínimo del nodo.



class OptimalValue:
    def __init__(self, node, optimal_cost):
        self.node = node;
        self.optimal_cost = optimal_cost;

In [19]:
#@markdown ## Implementacion recursiva del algoritmo Optimal BST

#@markdown ### Parametros

#@markdown `k`: (requerido) Arreglo de claves (k)eys 

#@markdown `d`: (requerido) Arreglo de probabilidades de las claves `k` 

#@markdown `f`: (requerido) Arreglo de probabilidades de las llaves falsas.

#@markdown `l`: Parametro Interno. Limite izquierdo del sub-arreglo 

#@markdown `r`: Parametro Interno. Limite derecho del sub-arreglo 

#@markdown `level`: Parametro Interno. Nivel de identacion de los prints

# Imprime cuatro espacios `level veces`

def printlevel(level):
    print("    "*level, end = "")

def optimal_bst(k, d, f, l = 0, r = -inf, level = 0):
    # Obtener largo automáticamente
    if r == -inf: r = len(k)-1


    printlevel(level);
    print("OBST:", d[l:r+1], f"l: {l};r: {r}")
    # Casos base
    if l>r:     # No hay elementos en el subarreglo
        printlevel(level);
        print ("no hay elementos.")
        return OptimalValue(node=None,optimal_cost=0)
    if l==r:
        printlevel(level);
        print (f"un elemento. [{d[l]}]")

        return OptimalValue(
                node=TreeNode(key = k[l], freq = d[l]),
                optimal_cost=d[l]
                )

    # Obtener la suma de los elementos
    fsum = sum(d[l:r+1])

    printlevel(level);
    print("suma:", fsum)
     
    # Inicializar el valor mínimo como el máximo
    # valor posible

    Min = inf
    nodo_salida = None

    
    # Uno por uno considerar todos los elementos como raíz y encontrar 
    # el costo del BST recursivamente comparar el costo con el mínimo
    # y actualizar de ser necesario.
    for m in range(l, int(r+1)):
        nodo = TreeNode(key = k[m], freq = d[m])
        printlevel(level)
        print(f"m={m}")
        # Subárbol izq
        printlevel(level)
        print("sub izq")
        
        ret_left = optimal_bst(k,d, l = l, r = m-1, level = level+1)
        if ret_left.node: nodo.append(ret_left.node)
        printlevel(level)
        print("fin izq")
        # Subárbol der
        printlevel(level)
        print("sub der")
        ret_right = optimal_bst(k,d, l = m+1, r = r, level = level+1)
        if ret_right.node: nodo.append(ret_right.node)
        printlevel(level)
        print("fin der")

        cost = ret_left.optimal_cost + ret_right.optimal_cost

        

        if cost < Min:
            printlevel(level)
            print (cost, "Es menor que", Min)
            Min = cost
            nodo_salida = nodo

    return OptimalValue(node = nodo_salida, optimal_cost=Min + fsum)

In [20]:
#@markdown ## Clase `Vector`
 
#@markdown Vector simple de dos dimensiones.

#@markdown ### Parámetros

#@markdown `x`: Coordenada x.

#@markdown `y`: Coordenada y.

class Vector:
  def __init__(self, x = 0, y = 0):
    self.x = x;
    self.y = y;

  def __str__(self):
    return f"({self.x}, {self.y})"

In [21]:
#@markdown ## Clase `Drawer`

#@markdown Grafica un (sub-)árbol dado una referencia `node` a un TreeNode.

#@markdown ### Parámetros

#@markdown `node`: (sub-)árbol a graficar.

#@markdown `filename`: (opcional) Nombre del archivo temporal

#@markdown `width`: (opcional) Ancho de la imagen de salida

#@markdown `height`: (opcional) Alto de la imagen de salida

#@markdown `radius`: (opcional) Radio de los nodos a dibujar

#@markdown `fontsize`: (opcional) Tamaño de letra de los nodos a dibujar

class Drawer:
  
  def __init__(self, node, filename = "tmp.svg", width = 1280, height = 720,
               radius = 35, fontsize = 16):
    self.__filename = filename
    self.__root = node  
    
    self.__width = width
    self.__height = height

    self.__radius = radius
    self.__fontsize = fontsize

    sep = width // 4

    self.__cursor = Vector(self.__width / 2, 10 + self.__radius)



    

    self.__surface = cairo.SVGSurface(self.__filename, self.__width,
                                      self.__height);

    self.__ctx = cairo.Context(self.__surface);

    self.__ctx.select_font_face("monospace")
    self.__ctx.set_source_rgb(0.0, 0.0, 0.0);
    self.__ctx.set_font_size(fontsize)
    self.__draw(node);
    self.__surface.finish()
    self.__show_svg(filename);
     
  def __update_cursor(self):
    self.__ctx.move_to(self.__cursor.x, self.__cursor.y)

  def __save_cursor(self):
    return deepcopy(self.__cursor)

  def __restore_cursor(self, cursor):
    self.__cursor = deepcopy(cursor)
    self.__update_cursor()

  def __move_radius_cursor(self, angle):
    self.__cursor.x += self.__radius * cos(angle * pi / 180)
    self.__cursor.y += self.__radius * sin(angle * pi / 180)
  
  
  def __draw(self, node, sep = -1):
    if node == None: return;
    if sep == -1: sep = self.__width // 4

    
    # dibujar nodo

    self.__ctx.arc(self.__cursor.x, self.__cursor.y, self.__radius, 0, 2*pi)
    self.__ctx.close_path()
    self.__ctx.set_source_rgb(0.0, 0.8, 0.0);
    self.__ctx.fill()
    self.__ctx.set_source_rgb(0.0, 0.0, 0.0);


    prev = self.__save_cursor() # Guardar posicion del cursor
    # print("prev: ", prev)


    # escribir texto del nodo
    
    e = self.__ctx.text_extents(f"{node.key}")
    self.__cursor.x -= e.width//2
    self.__cursor.y -= 10;
    self.__update_cursor();

    self.__ctx.show_text(f"{node.key}")
    self.__ctx.stroke()

    self.__cursor.x += e.width//2

    self.__cursor.y += e.height*2
    
    e = self.__ctx.text_extents(f"{node.freq}%")
    self.__cursor.x -= e.width//2

    self.__update_cursor();
    self.__ctx.show_text(f"{node.freq}%");
    self.__ctx.stroke();



    if node.left != None:
    # Recuperar cursor inicial
      self.__restore_cursor(prev)
      
    # Recuperar cursor inicial
      self.__move_radius_cursor(150);
      self.__update_cursor();

    # Configurar linea (color y ancho)
      self.__ctx.set_source_rgb(0, 0, 0)
      self.__ctx.set_line_width(0.5)

    # Trazar linea
      self.__ctx.line_to(self.__cursor.x - sep,
                        self.__cursor.y + 2*self.__radius)
      self.__ctx.stroke()

    # Mover cursor a la punta
      self.__cursor.x -= sep
      self.__cursor.y += 2*self.__radius;
    # Dibujar hijo izquierdo
      self.__draw(node.left, sep = sep // 1.68)
   

    


    if node.right != None:
    # Recuperar cursor inicial
      self.__restore_cursor(prev) 

    # Moverse 30 grados (hacia abajo ya que el eje y esta invertido)
      self.__move_radius_cursor(30); 
      self.__update_cursor();

    # Configurar linea
      self.__ctx.set_source_rgb(0, 0, 0)
      self.__ctx.set_line_width(0.5)

    # Trazar linea 
      self.__ctx.line_to(self.__cursor.x + sep,
                        self.__cursor.y + 2*self.__radius)
      self.__ctx.stroke()

    # Mover cursor a la punta
      self.__cursor.x += sep
      self.__cursor.y += 2*self.__radius;
      self.__update_cursor();
      
    # Dibujar hijo derecho
      self.__draw(node.right, sep = sep // 1.5)

  def __show_svg(self, file):
    display(SVG(filename=file))    



    


In [22]:
# t = TreeNode(key = 0)
optimal_value = optimal_bst([1,2,3,4],[5,10,30, 20], [35, 0, 0, 0, 0])
Drawer(optimal_value.node, width = 600, height = 300, radius = 20, fontsize = 12)
print(optimal_value.optimal_cost)

OBST: [5, 10, 30, 20] l: 0;r: 3
suma: 65
m=0
sub izq


TypeError: ignored